### importing libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
stemmer = WordNetLemmatizer()
import pickle
from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Mohamed
[nltk_data]     Mousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### load data

In [2]:
train = pd.read_csv('twitter_training.csv',sep=',', names=['Tweet_ID','Entity','Sentiment','Tweet_content'])
test = pd.read_csv('twitter_validation.csv',sep=',', names=['Tweet_ID','Entity','Sentiment','Tweet_content'])

In [3]:
train

,Tweet_ID,Entity,Sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


### CLEANING 

In [4]:
### drop duplicates
train.drop_duplicates(inplace=True)
### drop null values
train.dropna(inplace=True, axis=0)

In [5]:
### reset idx
train.reset_index(inplace=True)

In [6]:
### train_data_shape 
train.shape

(71656, 5)

In [7]:
### random sample from train data
train["Tweet_content"][90]

'I hate that this easy horrible modifier event on game 10 last forever. That is the best fun i’ve taken in the past until they added them horrible modifiers. @Borderlands please send me the option to play mayhem 10 but turn the modifiers for PLS'

In [8]:
### CLEANING TexT


stemmer = WordNetLemmatizer()

def clean_txt(x):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(x))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # word tokenize
    document = word_tokenize(document)
    # remove stop words
    document = [word for word in document if word not in stopwords.words('english')]
    # word lemmatization
    document = [stemmer.lemmatize(word) for word in document]
#      document = ' '.join(document)

    return document

In [9]:
train['Tweet_content'] = train['Tweet_content'].apply(lambda x:clean_txt(x))

In [10]:
### data after clean and tokenize
train["Tweet_content"][90]

['hate',
 'easy',
 'horrible',
 'modifier',
 'event',
 'game',
 '10',
 'last',
 'forever',
 'best',
 'fun',
 'taken',
 'past',
 'added',
 'horrible',
 'modifier',
 'borderland',
 'please',
 'send',
 'option',
 'play',
 'mayhem',
 '10',
 'turn',
 'modifier',
 'pls']

In [11]:
### save Train cleaned data
clean_train = train

In [12]:
test['Tweet_content'] = test['Tweet_content'].apply(lambda x:clean_txt(x))

In [13]:
### labeling for target data
size_dict = {'Positive':1, 
        'Neutral':2, 
        'Negative':3, 
        'Irrelevant':4}
train['ENC_Sentiment'] = train['Sentiment'].map(size_dict)
test['ENC_Sentiment'] = test['Sentiment'].map(size_dict)

In [14]:
### splitting data
X_train = train['Tweet_content']
y_train = train['ENC_Sentiment']
X_test = test['Tweet_content']
y_test = test['ENC_Sentiment']

### word2vec modeling

In [16]:
! pip install -U gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15


In [17]:
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
size = 100  # size of the vectors we want
window = 3  # number words before and after the focus word that it’ll consider as context for the word
min_count = 1  # the number of times a word must appear in our corpus in order to create a word vector.
workers = 3  # The number of partitions during training and the default workers is 3.
sg = 1  # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.

stemmed_tokens = pd.Series(X_train).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, vector_size = size, workers = workers, window = window, sg = sg)
word_vectors = w2v_model.wv
#w2v_model.save('vectors.kv')

In [18]:
print(word_vectors['hate'])
print('*'*25)
#  print(w2v_model.wv.index_to_key)
print('similar words of word love')
print('*'*25)
# Find the most similar words to "love" based on word vectors from our trained model
print(w2v_model.wv.most_similar('love'))

[-0.45890015  0.31556025  0.4592879   0.25051832  0.40044507 -0.06286726
  0.22624478  0.8604802  -0.75508285  0.28643885 -0.27832472 -0.45503384
  0.49393153 -0.436919    0.25125763  0.02613829 -0.58862936  0.3246154
  0.13046163 -0.5087403  -0.42129514 -0.21005532 -0.02071145  0.35756502
 -0.02694669  0.16860925  0.22884487 -0.26701796 -0.11317489 -0.02794731
 -0.1537673  -0.91712     1.2012904  -0.0528184  -0.22897635  0.05656347
  0.54949385 -0.0357813   0.28037393 -0.72820985  0.57580525 -0.26953933
 -0.20112498  0.26658428  0.27186587 -0.3157961  -0.2241736  -0.5037487
 -0.00933324 -0.11542544  0.73735     0.36785465  0.2392153   0.24950153
 -0.14556883 -0.1696968  -0.2950396   0.6010623  -0.50673306 -0.32036516
 -0.25960615  0.5862375   0.394842   -1.3282658  -0.16954167  0.35974252
  0.55754876  0.43181673 -0.17275335  0.37526932  0.22101785  0.10353065
  0.24805532  0.43499663  0.5606754   0.36633217 -0.02086623 -0.08182224
 -0.13361517  0.19217493 -0.07695284  0.49241546  0.3

In [19]:
### Generate aggregated sentence vectors based on the word vectors for each word in the sentence
import numpy as np
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in stemmed_tokens])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [20]:
# Why is the length of the sentence different than the length of the sentence vector?
for i, v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]), len(v))

4 4
3 3
4 4
4 4
5 5
4 4
27 27
27 27
12 12
27 27
28 28
0 0
13 13
13 13
13 13
13 13
18 18
12 12
14 14
14 14
15 15
17 17
13 13
9 9
6 6
9 9
12 12
11 11
24 24
23 23
22 22
24 24
30 30
23 23
30 30
23 23
29 29
28 28
33 33
28 28
3 3
3 3
3 3
3 3
4 4
4 4
18 18
12 12
9 9
18 18
0 0
11 11
9 9
10 10
11 11
14 14
11 11
0 0
0 0
1 1
0 0
15 15
13 13
12 12
16 16
17 17
15 15
7 7
7 7
8 8
8 8
8 8
8 8
17 17
20 20
20 20
16 16
23 23
16 16
2 2
2 2
2 2
3 3
2 2
1 1
24 24
23 23
24 24
25 25
31 31
26 26
22 22
17 17
17 17
20 20
28 28
21 21
23 23
22 22
22 22
22 22
31 31
24 24
7 7
7 7
7 7
7 7
10 10
6 6
3 3
3 3
3 3
4 4
4 4
3 3
26 26
23 23
7 7
27 27
35 35
27 27
8 8
8 8
8 8
8 8
11 11
6 6
6 6
5 5
6 6
5 5
7 7
6 6
4 4
4 4
4 4
5 5
6 6
4 4
1 1
1 1
2 2
1 1
1 1
13 13
13 13
12 12
13 13
15 15
13 13
6 6
6 6
6 6
6 6
7 7
6 6
13 13
12 12
7 7
13 13
16 16
14 14
7 7
7 7
7 7
8 8
7 7
4 4
4 4
4 4
4 4
6 6
3 3
19 19
18 18
18 18
20 20
25 25
19 19
29 29
30 30
25 25
29 29
0 0
5 5
4 4
6 6
5 5
5 5
5 5
5 5
5 5
6 6
0 0
4 4
4 4
5 5
3 3
6 6
6 6
6 6
6 6


4 4
4 4
2 2
4 4
5 5
4 4
15 15
14 14
13 13
16 16
17 17
14 14
5 5
6 6
5 5
5 5
6 6
6 6
19 19
16 16
10 10
19 19
24 24
20 20
17 17
16 16
15 15
17 17
23 23
14 14
4 4
4 4
4 4
5 5
1 1
16 16
15 15
14 14
16 16
20 20
16 16
12 12
11 11
6 6
11 11
15 15
13 13
6 6
8 8
6 6
6 6
7 7
8 8
13 13
13 13
13 13
14 14
13 13
13 13
19 19
18 18
18 18
20 20
26 26
21 21
15 15
15 15
15 15
19 19
19 19
15 15
11 11
8 8
8 8
10 10
14 14
11 11
21 21
19 19
21 21
22 22
24 24
24 24
10 10
11 11
7 7
10 10
1 1
13 13
11 11
13 13
13 13
17 17
12 12
16 16
18 18
198 198
17 17
25 25
16 16
13 13
12 12
13 13
13 13
17 17
13 13
27 27
30 30
28 28
27 27
34 34
30 30
19 19
20 20
15 15
21 21
25 25
19 19
5 5
6 6
7 7
5 5
7 7
6 6
15 15
14 14
17 17
16 16
0 0
11 11
10 10
6 6
11 11
11 11
0 0
4 4
4 4
5 5
4 4
5 5
4 4
11 11
8 8
11 11
12 12
12 12
9 9
27 27
29 29
28 28
30 30
34 34
28 28
15 15
14 14
13 13
15 15
18 18
17 17
21 21
20 20
20 20
22 22
26 26
19 19
1 1
1 1
2 2
2 2
2 2
1 1
3 3
3 3
2 2
3 3
3 3
3 3
27 27
21 21
27 27
28 28
37 37
30 30
9 9
9 9
7 7
8 

2 2
3 3
2 2
15 15
15 15
15 15
16 16
21 21
16 16
21 21
18 18
21 21
22 22
26 26
21 21
3 3
3 3
3 3
3 3
4 4
3 3
9 9
9 9
9 9
10 10
12 12
8 8
0 0
11 11
0 0
0 0
22 22
22 22
22 22
23 23
28 28
22 22
3 3
1 1
1 1
3 3
3 3
2 2
5 5
5 5
5 5
5 5
8 8
6 6
1 1
1 1
1 1
1 1
2 2
16 16
16 16
15 15
16 16
21 21
16 16
14 14
14 14
14 14
15 15
19 19
18 18
9 9
9 9
10 10
9 9
10 10
10 10
5 5
5 5
3 3
4 4
5 5
5 5
1 1
1 1
1 1
1 1
1 1
0 0
17 17
15 15
17 17
18 18
21 21
15 15
3 3
3 3
3 3
3 3
4 4
4 4
2 2
2 2
2 2
2 2
2 2
6 6
7 7
7 7
6 6
9 9
6 6
7 7
6 6
7 7
10 10
6 6
10 10
10 10
10 10
11 11
13 13
10 10
3 3
3 3
5 5
3 3
4 4
3 3
2 2
2 2
2 2
2 2
2 2
1 1
1 1
27 27
2 2
2 2
1 1
2 2
1 1
2 2
2 2
2 2
7 7
7 7
6 6
8 8
8 8
6 6
24 24
24 24
22 22
25 25
29 29
22 22
15 15
10 10
14 14
14 14
19 19
16 16
4 4
4 4
4 4
4 4
3 3
9 9
11 11
11 11
10 10
13 13
10 10
14 14
14 14
14 14
15 15
18 18
14 14
7 7
6 6
6 6
7 7
9 9
7 7
9 9
5 5
4 4
9 9
12 12
10 10
18 18
14 14
15 15
19 19
0 0
22 22
19 19
8 8
22 22
26 26
21 21
8 8
8 8
8 8
8 8
10 10
8 8
3 3
3 3
3 3
3 

30 30
27 27
29 29
30 30
38 38
30 30
16 16
16 16
7 7
16 16
21 21
17 17
16 16
15 15
17 17
22 22
17 17
0 0
6 6
19 19
7 7
5 5
7 7
8 8
10 10
9 9
27 27
26 26
11 11
27 27
27 27
0 0
13 13
12 12
10 10
13 13
15 15
13 13
20 20
20 20
6 6
19 19
25 25
17 17
14 14
14 14
13 13
14 14
19 19
13 13
16 16
16 16
10 10
16 16
20 20
16 16
30 30
28 28
4 4
29 29
36 36
31 31
18 18
17 17
16 16
19 19
25 25
19 19
12 12
11 11
10 10
13 13
14 14
12 12
14 14
14 14
14 14
15 15
22 22
16 16
25 25
26 26
10 10
26 26
30 30
28 28
14 14
14 14
14 14
14 14
17 17
14 14
8 8
7 7
7 7
10 10
11 11
9 9
3 3
3 3
3 3
3 3
3 3
1 1
1 1
2 2
0 0
22 22
27 27
21 21
22 22
22 22
0 0
1 1
1 1
1 1
1 1
0 0
27 27
29 29
27 27
27 27
35 35
28 28
5 5
5 5
6 6
4 4
7 7
5 5
8 8
7 7
7 7
8 8
12 12
9 9
11 11
11 11
10 10
12 12
15 15
13 13
26 26
26 26
26 26
27 27
0 0
0 0
6 6
19 19
1 1
1 1
1 1
1 1
2 2
3 3
12 12
12 12
9 9
15 15
16 16
13 13
11 11
11 11
6 6
11 11
12 12
11 11
14 14
14 14
14 14
14 14
16 16
15 15
11 11
12 12
9 9
13 13
15 15
10 10
14 14
14 14
14 14
13 13
17

31 31
23 23
17 17
15 15
14 14
16 16
23 23
19 19
6 6
6 6
6 6
7 7
0 0
6 6
5 5
6 6
9 9
8 8
8 8
8 8
7 7
8 8
10 10
8 8
13 13
10 10
12 12
13 13
0 0
5 5
5 5
5 5
6 6
8 8
5 5
7 7
7 7
7 7
7 7
0 0
1 1
1 1
2 2
1 1
1 1
9 9
9 9
9 9
9 9
11 11
9 9
10 10
10 10
9 9
13 13
12 12
11 11
4 4
4 4
4 4
4 4
5 5
4 4
5 5
4 4
5 5
5 5
6 6
4 4
28 28
17 17
17 17
28 28
35 35
29 29
6 6
6 6
5 5
6 6
6 6
5 5
3 3
2 2
2 2
3 3
5 5
4 4
8 8
5 5
8 8
8 8
8 8
8 8
12 12
12 12
10 10
12 12
14 14
10 10
35 35
34 34
32 32
36 36
39 39
35 35
6 6
4 4
4 4
8 8
9 9
7 7
22 22
20 20
18 18
19 19
25 25
21 21
4 4
4 4
3 3
5 5
4 4
3 3
10 10
10 10
10 10
10 10
0 0
16 16
13 13
15 15
16 16
21 21
17 17
10 10
9 9
3 3
10 10
0 0
11 11
13 13
13 13
11 11
12 12
0 0
16 16
16 16
13 13
16 16
20 20
14 14
16 16
16 16
16 16
17 17
22 22
18 18
4 4
4 4
3 3
4 4
4 4
5 5
1 1
1 1
2 2
2 2
3 3
1 1
2 2
2 2
1 1
12 12
10 10
8 8
12 12
13 13
13 13
1 1
1 1
1 1
1 1
1 1
16 16
16 16
5 5
14 14
14 14
0 0
6 6
5 5
6 6
6 6
7 7
0 0
23 23
22 22
24 24
26 26
27 27
22 22
18 18
17 17
15 15
18 1

...

13 13
8 8
2 2
3 3
3 3
2 2
3 3
3 3
2 2
2 2
3 3
3 3
12 12
12 12
8 8
12 12
17 17
13 13
15 15
15 15
15 15
17 17
19 19
16 16
6 6
6 6
5 5
5 5
8 8
5 5
1 1
1 1
1 1
1 1
7 7
7 7
8 8
7 7
12 12
12 12
11 11
13 13
16 16
13 13
5 5
5 5
3 3
4 4
5 5
5 5
7 7
7 7
7 7
7 7
10 10
7 7
18 18
16 16
17 17
18 18
22 22
19 19
7 7
3 3
7 7
7 7
7 7
9 9
3 3
3 3
3 3
4 4
3 3
4 4
6 6
6 6
6 6
6 6
6 6
7 7
8 8
8 8
9 9
8 8
11 11
8 8
4 4
3 3
4 4
5 5
5 5
4 4
7 7
4 4
7 7
7 7
8 8
5 5
7 7
9 9
9 9
8 8
8 8
5 5
2 2
2 2
2 2
3 3
2 2
3 3
3 3
3 3
3 3
3 3
4 4
10 10
8 8
9 9
11 11
13 13
13 13
11 11
13 13
12 12
11 11
15 15
10 10
22 22
20 20
15 15
22 22
23 23
0 0
5 5
5 5
5 5
5 5
5 5
5 5
3 3
3 3
3 3
4 4
3 3
7 7
7 7
5 5
7 7
7 7
0 0
4 4
3 3
3 3
4 4
5 5
5 5
0 0
6 6
19 19
1 1
1 1
2 2
0 0
8 8
8 8
8 8
8 8
10 10
6 6
3 3
3 3
3 3
3 3
4 4
3 3
12 12
12 12
12 12
12 12
17 17
11 11
5 5
4 4
5 5
5 5
6 6
5 5
15 15
15 15
16 16
14 14
19 19
13 13
23 23
19 19
11 11
25 25
31 31
23 23
3 3
2 2
3 3
4 4
5 5
4 4
10 10
9 9
9 9
10 10
14 14
10 10
5 5
4 4
5 5
5 5
7 7
5 5
31

22 22
35 35
32 32
25 25
34 34
43 43
34 34
4 4
4 4
4 4
3 3
4 4
4 4
13 13
13 13
12 12
13 13
18 18
11 11
12 12
11 11
12 12
14 14
14 14
14 14
21 21
14 14
19 19
22 22
27 27
21 21
3 3
2 2
2 2
3 3
3 3
3 3
27 27
27 27
21 21
27 27
35 35
25 25
23 23
21 21
14 14
22 22
27 27
21 21
8 8
8 8
6 6
8 8
10 10
8 8
19 19
21 21
19 19
19 19
21 21
18 18
3 3
2 2
4 4
3 3
2 2
22 22
21 21
17 17
23 23
28 28
20 20
24 24
23 23
11 11
25 25
31 31
26 26
16 16
16 16
16 16
15 15
22 22
16 16
11 11
10 10
11 11
13 13
12 12
10 10
33 33
31 31
17 17
35 35
42 42
35 35
11 11
11 11
11 11
11 11
14 14
11 11
14 14
14 14
15 15
14 14
19 19
15 15
34 34
23 23
36 36
35 35
41 41
38 38
7 7
7 7
3 3
7 7
10 10
7 7
15 15
15 15
13 13
15 15
18 18
14 14
9 9
9 9
10 10
9 9
13 13
12 12
13 13
12 12
16 16
13 13
16 16
15 15
10 10
17 17
20 20
16 16
3 3
3 3
2 2
3 3
3 3
13 13
13 13
9 9
13 13
17 17
13 13
7 7
7 7
7 7
9 9
8 8
23 23
23 23
23 23
26 26
31 31
24 24
3 3
2 2
3 3
3 3
4 4
3 3
4 4
5 5
4 4
4 4
6 6
4 4
1 1
1 1
2 2
0 0
1 1
1 1
15 15
15 15
11 11
15 15
19

2 2
11 11
11 11
11 11
11 11
14 14
11 11
25 25
22 22
21 21
27 27
28 28
26 26
3 3
3 3
3 3
4 4
3 3
3 3
7 7
6 6
6 6
8 8
10 10
7 7
1 1
1 1
1 1
2 2
2 2
1 1
0 0
6 6
19 19
26 26
21 21
13 13
26 26
35 35
27 27
8 8
8 8
8 8
8 8
11 11
10 10
6 6
6 6
6 6
6 6
1 1
7 7
7 7
8 8
6 6
9 9
5 5
6 6
5 5
6 6
6 6
7 7
6 6
28 28
26 26
28 28
28 28
32 32
25 25
7 7
7 7
7 7
8 8
9 9
7 7
1 1
1 1
1 1
0 0
1 1
1 1
10 10
11 11
9 9
12 12
13 13
10 10
6 6
6 6
5 5
6 6
7 7
5 5
23 23
18 18
19 19
24 24
32 32
24 24
6 6
6 6
5 5
7 7
9 9
6 6
5 5
4 4
4 4
5 5
6 6
4 4
4 4
5 5
5 5
5 5
7 7
4 4
20 20
20 20
9 9
18 18
28 28
19 19
8 8
8 8
8 8
7 7
13 13
8 8
3 3
4 4
5 5
3 3
7 7
7 7
7 7
7 7
7 7
7 7
2 2
2 2
2 2
2 2
2 2
1 1
6 6
6 6
3 3
6 6
8 8
6 6
12 12
12 12
12 12
12 12
14 14
15 15
4 4
4 4
4 4
4 4
5 5
4 4
3 3
3 3
4 4
3 3
3 3
3 3
17 17
17 17
13 13
17 17
18 18
0 0
1 1
1 1
1 1
1 1
2 2
1 1
30 30
26 26
26 26
29 29
34 34
31 31
4 4
4 4
4 4
5 5
6 6
4 4
9 9
9 9
9 9
9 9
0 0
6 6
7 7
4 4
6 6
8 8
8 8
2 2
2 2
3 3
4 4
2 2
12 12
12 12
14 14
12 12
17 17
12 12
3 3


8 8
9 9
10 10
7 7
2 2
2 2
2 2
2 2
4 4
2 2
26 26
26 26
27 27
26 26
30 30
22 22
27 27
25 25
27 27
28 28
31 31
27 27
1 1
1 1
1 1
2 2
1 1
4 4
5 5
5 5
5 5
7 7
4 4
12 12
14 14
14 14
13 13
17 17
14 14
6 6
6 6
6 6
6 6
7 7
5 5
3 3
3 3
3 3
4 4
3 3
3 3
25 25
24 24
25 25
27 27
28 28
27 27
25 25
25 25
12 12
27 27
30 30
28 28
4 4
6 6
5 5
4 4
7 7
5 5
3 3
4 4
3 3
4 4
3 3
2 2
1 1
1 1
2 2
1 1
1 1
0 0
21 21
20 20
13 13
21 21
28 28
21 21
2 2
1 1
2 2
2 2
1 1
4 4
6 6
5 5
5 5
4 4
3 3
3 3
3 3
3 3
4 4
3 3
25 25
21 21
19 19
24 24
27 27
27 27
12 12
12 12
12 12
10 10
12 12
8 8
8 8
6 6
5 5
8 8
10 10
8 8
10 10
11 11
11 11
10 10
13 13
10 10
8 8
12 12
7 7
8 8
10 10
8 8
2 2
2 2
132 132
3 3
3 3
2 2
2 2
2 2
2 2
2 2
3 3
1 1
12 12
13 13
14 14
12 12
17 17
12 12
30 30
29 29
30 30
32 32
36 36
31 31
22 22
23 23
28 28
22 22
23 23
0 0
8 8
10 10
9 9
8 8
9 9
0 0
1 1
2 2
1 1
1 1
2 2
0 0
9 9
9 9
9 9
10 10
10 10
8 8
14 14
15 15
14 14
15 15
19 19
15 15
9 9
9 9
4 4
10 10
11 11
9 9
13 13
15 15
15 15
13 13
14 14
1 1
30 30
30 30
30 30
30

29 29
25 25
32 32
35 35
30 30
12 12
12 12
10 10
14 14
18 18
11 11
8 8
8 8
8 8
9 9
10 10
8 8
25 25
25 25
24 24
27 27
33 33
27 27
13 13
13 13
13 13
15 15
18 18
13 13
0 0
6 6
19 19
9 9
11 11
9 9
8 8
12 12
7 7
8 8
8 8
8 8
8 8
9 9
7 7
31 31
31 31
30 30
32 32
35 35
31 31
11 11
11 11
11 11
11 11
13 13
11 11
6 6
5 5
6 6
6 6
7 7
5 5
14 14
14 14
8 8
14 14
20 20
14 14
18 18
18 18
18 18
20 20
24 24
19 19
29 29
23 23
29 29
32 32
32 32
30 30
2 2
2 2
2 2
2 2
3 3
13 13
13 13
13 13
14 14
18 18
15 15
13 13
12 12
13 13
13 13
14 14
14 14
0 0
6 6
19 19
12 12
13 13
12 12
10 10
13 13
14 14
5 5
4 4
5 5
5 5
4 4
17 17
17 17
16 16
16 16
22 22
17 17
35 35
34 34
14 14
37 37
39 39
34 34
20 20
19 19
20 20
21 21
24 24
24 24
33 33
32 32
27 27
32 32
41 41
32 32
12 12
12 12
12 12
12 12
16 16
11 11
4 4
4 4
5 5
4 4
6 6
4 4
10 10
9 9
9 9
10 10
12 12
10 10
2 2
2 2
2 2
2 2
2 2
1 1
1 1
1 1
1 1
1 1
18 18
14 14
13 13
19 19
27 27
19 19
6 6
6 6
6 6
7 7
7 7
5 5
42 42
41 41
38 38
43 43
51 51
43 43
16 16
18 18
19 19
17 17
0 0
14 14


In [21]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [22]:
# Are our sentence vector lengths consistent?
for i, v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]), len(v))

4 100
3 100
4 100
4 100
5 100
4 100
27 100
27 100
12 100
27 100
28 100
0 100
13 100
13 100
13 100
13 100
18 100
12 100
14 100
14 100
15 100
17 100
13 100
9 100
6 100
9 100
12 100
11 100
24 100
23 100
22 100
24 100
30 100
23 100
30 100
23 100
29 100
28 100
33 100
28 100
3 100
3 100
3 100
3 100
4 100
4 100
18 100
12 100
9 100
18 100
0 100
11 100
9 100
10 100
11 100
14 100
11 100
0 100
0 100
1 100
0 100
15 100
13 100
12 100
16 100
17 100
15 100
7 100
7 100
8 100
8 100
8 100
8 100
17 100
20 100
20 100
16 100
23 100
16 100
2 100
2 100
2 100
3 100
2 100
1 100
24 100
23 100
24 100
25 100
31 100
26 100
22 100
17 100
17 100
20 100
28 100
21 100
23 100
22 100
22 100
22 100
31 100
24 100
7 100
7 100
7 100
7 100
10 100
6 100
3 100
3 100
3 100
4 100
4 100
3 100
26 100
23 100
7 100
27 100
35 100
27 100
8 100
8 100
8 100
8 100
11 100
6 100
6 100
5 100
6 100
5 100
7 100
6 100
4 100
4 100
4 100
5 100
6 100
4 100
1 100
1 100
2 100
1 100
1 100
13 100
13 100
12 100
13 100
15 100
13 100
6 100
6 100
6 100
6

22 100
24 100
24 100
10 100
11 100
7 100
10 100
1 100
13 100
11 100
13 100
13 100
17 100
12 100
16 100
18 100
198 100
17 100
25 100
16 100
13 100
12 100
13 100
13 100
17 100
13 100
27 100
30 100
28 100
27 100
34 100
30 100
19 100
20 100
15 100
21 100
25 100
19 100
5 100
6 100
7 100
5 100
7 100
6 100
15 100
14 100
17 100
16 100
0 100
11 100
10 100
6 100
11 100
11 100
0 100
4 100
4 100
5 100
4 100
5 100
4 100
11 100
8 100
11 100
12 100
12 100
9 100
27 100
29 100
28 100
30 100
34 100
28 100
15 100
14 100
13 100
15 100
18 100
17 100
21 100
20 100
20 100
22 100
26 100
19 100
1 100
1 100
2 100
2 100
2 100
1 100
3 100
3 100
2 100
3 100
3 100
3 100
27 100
21 100
27 100
28 100
37 100
30 100
9 100
9 100
7 100
8 100
11 100
9 100
22 100
22 100
22 100
23 100
0 100
6 100
6 100
5 100
6 100
7 100
6 100
25 100
24 100
23 100
26 100
32 100
26 100
11 100
11 100
6 100
10 100
14 100
11 100
10 100
10 100
10 100
10 100
10 100
10 100
4 100
3 100
4 100
4 100
5 100
4 100
3 100
3 100
4 100
3 100
23 100
21 100
24 

3 100
3 100
3 100
3 100
3 100
3 100
16 100
16 100
14 100
14 100
21 100
17 100
4 100
4 100
4 100
6 100
4 100
21 100
20 100
11 100
22 100
27 100
23 100
9 100
9 100
8 100
9 100
11 100
9 100
3 100
4 100
4 100
3 100
3 100
3 100
4 100
4 100
4 100
4 100
5 100
4 100
19 100
18 100
19 100
19 100
19 100
0 100
4 100
4 100
3 100
4 100
3 100
23 100
20 100
21 100
25 100
29 100
25 100
1 100
1 100
2 100
1 100
10 100
11 100
5 100
10 100
0 100
7 100
9 100
6 100
8 100
6 100
3 100
2 100
2 100
3 100
3 100
3 100
3 100
3 100
3 100
3 100
10 100
9 100
9 100
10 100
12 100
10 100
16 100
18 100
17 100
16 100
21 100
15 100
16 100
13 100
14 100
16 100
24 100
17 100
9 100
8 100
8 100
9 100
10 100
7 100
7 100
7 100
8 100
7 100
9 100
8 100
1 100
2 100
1 100
1 100
1 100
1 100
10 100
8 100
9 100
10 100
14 100
9 100
5 100
5 100
4 100
5 100
8 100
5 100
6 100
3 100
3 100
6 100
6 100
6 100
4 100
5 100
4 100
4 100
5 100
4 100
6 100
5 100
6 100
7 100
6 100
6 100
5 100
6 100
8 100
7 100
0 100
6 100
19 100
8 100
5 100
7 100
9 10

30 100
30 100
30 100
37 100
28 100
3 100
3 100
3 100
4 100
3 100
3 100
14 100
14 100
14 100
14 100
18 100
12 100
30 100
30 100
28 100
28 100
38 100
27 100
2 100
2 100
3 100
2 100
3 100
2 100
14 100
13 100
14 100
15 100
21 100
16 100
14 100
14 100
13 100
14 100
17 100
15 100
12 100
12 100
12 100
12 100
12 100
12 100
9 100
9 100
8 100
9 100
10 100
9 100
10 100
9 100
9 100
10 100
12 100
10 100
23 100
22 100
21 100
22 100
28 100
24 100
14 100
14 100
14 100
14 100
17 100
16 100
14 100
13 100
13 100
14 100
18 100
16 100
6 100
6 100
6 100
7 100
7 100
6 100
16 100
16 100
17 100
16 100
24 100
16 100
30 100
30 100
8 100
28 100
36 100
30 100
4 100
5 100
4 100
4 100
4 100
4 100
23 100
25 100
16 100
25 100
31 100
23 100
8 100
8 100
51 100
7 100
10 100
7 100
14 100
14 100
14 100
14 100
17 100
14 100
30 100
28 100
26 100
31 100
33 100
35 100
28 100
27 100
27 100
31 100
35 100
30 100
14 100
13 100
13 100
13 100
17 100
16 100
6 100
6 100
6 100
5 100
6 100
7 100
22 100
21 100
10 100
23 100
26 100
22 100

5 100
7 100
6 100
5 100
3 100
2 100
3 100
2 100
23 100
22 100
17 100
24 100
27 100
21 100
29 100
28 100
12 100
29 100
32 100
32 100
7 100
6 100
6 100
7 100
9 100
7 100
2 100
2 100
2 100
2 100
2 100
5 100
5 100
5 100
5 100
5 100
4 100
2 100
3 100
3 100
2 100
2 100
3 100
16 100
14 100
12 100
16 100
18 100
14 100
5 100
5 100
5 100
6 100
4 100
8 100
5 100
8 100
7 100
8 100
6 100
11 100
11 100
7 100
10 100
12 100
12 100
11 100
11 100
11 100
12 100
12 100
13 100
1 100
1 100
1 100
2 100
1 100
7 100
7 100
7 100
7 100
8 100
6 100
8 100
8 100
7 100
8 100
8 100
7 100
10 100
9 100
10 100
9 100
13 100
9 100
2 100
2 100
2 100
3 100
2 100
3 100
2 100
2 100
2 100
4 100
2 100
4 100
4 100
3 100
4 100
3 100
3 100
3 100
3 100
4 100
5 100
4 100
3 100
3 100
3 100
3 100
5 100
3 100
14 100
11 100
12 100
13 100
19 100
15 100
14 100
14 100
14 100
14 100
17 100
14 100
33 100
33 100
32 100
31 100
38 100
30 100
8 100
7 100
8 100
8 100
8 100
9 100
23 100
21 100
23 100
25 100
30 100
24 100
27 100
27 100
26 100
27 10

...

2 100
4 100
6 100
6 100
4 100
5 100
4 100
17 100
16 100
16 100
17 100
19 100
17 100
3 100
3 100
3 100
4 100
4 100
3 100
12 100
11 100
12 100
12 100
14 100
12 100
7 100
6 100
7 100
8 100
7 100
6 100
5 100
6 100
7 100
8 100
7 100
7 100
6 100
7 100
7 100
8 100
7 100
5 100
5 100
5 100
6 100
5 100
8 100
10 100
10 100
9 100
11 100
11 100
24 100
24 100
16 100
25 100
27 100
26 100
9 100
8 100
11 100
8 100
12 100
9 100
28 100
28 100
17 100
29 100
34 100
29 100
5 100
5 100
5 100
6 100
6 100
4 100
7 100
6 100
7 100
7 100
11 100
8 100
4 100
4 100
4 100
4 100
5 100
4 100
3 100
3 100
4 100
3 100
3 100
5 100
4 100
4 100
6 100
5 100
5 100
3 100
3 100
5 100
6 100
5 100
16 100
16 100
14 100
16 100
17 100
0 100
5 100
5 100
5 100
5 100
7 100
6 100
4 100
4 100
4 100
6 100
4 100
2 100
2 100
2 100
2 100
2 100
3 100
17 100
17 100
17 100
18 100
22 100
17 100
3 100
2 100
4 100
4 100
2 100
14 100
14 100
13 100
16 100
18 100
15 100
18 100
18 100
18 100
18 100
18 100
0 100
13 100
13 100
13 100
14 100
19 100
14 100

23 100
18 100
27 100
27 100
28 100
29 100
32 100
27 100
34 100
33 100
13 100
36 100
41 100
32 100
13 100
11 100
10 100
14 100
17 100
13 100
10 100
10 100
10 100
10 100
11 100
7 100
8 100
8 100
8 100
9 100
11 100
7 100
13 100
13 100
13 100
13 100
18 100
11 100
3 100
3 100
3 100
4 100
3 100
19 100
17 100
20 100
20 100
23 100
20 100
18 100
17 100
13 100
18 100
20 100
17 100
6 100
6 100
6 100
6 100
8 100
6 100
29 100
28 100
27 100
29 100
34 100
27 100
21 100
21 100
23 100
21 100
26 100
20 100
2 100
2 100
3 100
2 100
24 100
23 100
24 100
25 100
33 100
26 100
17 100
18 100
9 100
17 100
25 100
18 100
3 100
3 100
4 100
3 100
3 100
3 100
4 100
4 100
4 100
4 100
5 100
3 100
6 100
6 100
6 100
6 100
8 100
6 100
13 100
9 100
12 100
13 100
17 100
14 100
10 100
10 100
8 100
10 100
15 100
12 100
17 100
18 100
17 100
19 100
23 100
17 100
9 100
8 100
9 100
9 100
10 100
8 100
24 100
23 100
23 100
26 100
30 100
21 100
6 100
8 100
6 100
6 100
10 100
6 100
10 100
9 100
9 100
9 100
11 100
10 100
21 100
22 10

4 100
7 100
6 100
3 100
6 100
8 100
7 100
8 100
7 100
10 100
7 100
9 100
9 100
1 100
1 100
1 100
2 100
1 100
0 100
17 100
16 100
11 100
17 100
22 100
18 100
1 100
2 100
10 100
10 100
9 100
11 100
0 100
10 100
9 100
10 100
11 100
11 100
11 100
19 100
19 100
15 100
18 100
26 100
18 100
27 100
29 100
1 100
32 100
37 100
27 100
29 100
26 100
21 100
28 100
35 100
30 100
12 100
11 100
7 100
12 100
15 100
13 100
21 100
15 100
10 100
22 100
26 100
22 100
3 100
3 100
3 100
4 100
5 100
3 100
33 100
32 100
19 100
32 100
39 100
33 100
18 100
18 100
19 100
18 100
22 100
20 100
0 100
6 100
19 100
3 100
3 100
3 100
4 100
4 100
3 100
3 100
3 100
4 100
4 100
4 100
3 100
6 100
6 100
6 100
7 100
7 100
17 100
19 100
18 100
16 100
24 100
19 100
4 100
4 100
4 100
4 100
5 100
4 100
6 100
6 100
5 100
7 100
7 100
6 100
7 100
7 100
7 100
9 100
9 100
6 100
4 100
4 100
4 100
5 100
5 100
6 100
6 100
6 100
6 100
9 100
7 100
21 100
19 100
21 100
23 100
26 100
22 100
16 100
13 100
12 100
16 100
22 100
17 100
1 100
1 

4 100
6 100
3 100
25 100
23 100
20 100
26 100
30 100
24 100
21 100
12 100
12 100
22 100
27 100
24 100
21 100
20 100
18 100
21 100
22 100
0 100
10 100
9 100
8 100
10 100
12 100
11 100
1 100
1 100
2 100
1 100
1 100
2 100
17 100
18 100
8 100
16 100
23 100
17 100
31 100
30 100
31 100
27 100
36 100
32 100
7 100
5 100
6 100
7 100
10 100
7 100
13 100
13 100
11 100
13 100
20 100
14 100
24 100
24 100
8 100
24 100
30 100
22 100
13 100
13 100
13 100
14 100
17 100
14 100
22 100
20 100
21 100
21 100
27 100
26 100
10 100
10 100
10 100
10 100
10 100
11 100
9 100
9 100
10 100
10 100
11 100
8 100
1 100
2 100
1 100
0 100
1 100
2 100
2 100
2 100
1 100
2 100
2 100
1 100
6 100
6 100
5 100
6 100
7 100
0 100
5 100
5 100
5 100
5 100
8 100
5 100
3 100
3 100
4 100
3 100
2 100
15 100
11 100
13 100
15 100
20 100
13 100
9 100
9 100
9 100
9 100
11 100
10 100
9 100
9 100
9 100
9 100
11 100
9 100
10 100
10 100
11 100
10 100
13 100
10 100
14 100
14 100
11 100
15 100
18 100
16 100
0 100
6 100
19 100
7 100
7 100
5 100
7

14 100
29 100
24 100
10 100
30 100
35 100
31 100
20 100
20 100
20 100
19 100
25 100
20 100
24 100
22 100
25 100
26 100
27 100
22 100
6 100
6 100
5 100
6 100
8 100
6 100
16 100
16 100
18 100
15 100
18 100
17 100
18 100
18 100
7 100
18 100
21 100
18 100
10 100
10 100
9 100
10 100
15 100
11 100
8 100
7 100
10 100
8 100
11 100
9 100
38 100
42 100
35 100
30 100
41 100
33 100
25 100
17 100
19 100
26 100
31 100
27 100
14 100
14 100
14 100
14 100
18 100
15 100
0 100
6 100
19 100
11 100
11 100
12 100
14 100
11 100
2 100
2 100
2 100
1 100
2 100
1 100
19 100
17 100
17 100
19 100
20 100
0 100
25 100
26 100
26 100
27 100
32 100
26 100
9 100
9 100
9 100
13 100
9 100
23 100
22 100
22 100
24 100
31 100
23 100
2 100
2 100
3 100
3 100
2 100
10 100
11 100
11 100
11 100
13 100
9 100
1 100
1 100
1 100
0 100
6 100
6 100
6 100
7 100
8 100
7 100
5 100
5 100
4 100
5 100
5 100
6 100
27 100
31 100
10 100
28 100
0 100
4 100
4 100
3 100
3 100
5 100
3 100
18 100
19 100
13 100
18 100
21 100
18 100
9 100
9 100
9 100


### ML modeling

In [23]:
### Fit RandomForestClassifier On Top Of Word Vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values)

In [24]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

In [25]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred) ## confusion matrix
cr = classification_report(y_test,y_pred) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)

Confusion matrix

 [[256  11   8   2]
 [ 18 258   7   2]
 [  8   4 254   0]
 [ 13  12   7 140]] 

              precision    recall  f1-score   support

           1       0.87      0.92      0.90       277
           2       0.91      0.91      0.91       285
           3       0.92      0.95      0.94       266
           4       0.97      0.81      0.89       172

    accuracy                           0.91      1000
   macro avg       0.92      0.90      0.91      1000
weighted avg       0.91      0.91      0.91      1000



In [29]:
# predction = rf_model.predict('I hate the selfishness in you')[0]
# print(predction)